In [1]:
import warnings
warnings.filterwarnings('ignore')

import mercury as mr
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dill
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.neighbors import KNeighborsClassifier

plt.rcParams['figure.figsize'] = (8,5)
plt.rcParams['font.size'] = 14

In [2]:
file = "arxiv-metadata-oai-snapshot.json"
def get_data():
    with open(file) as f:
        for line in f:
            yield line

In [3]:
dataframe = {
    'title': [],
    'year': [],
    'abstract': []
}

data = get_data()
for i, paper in enumerate(data):
    paper = json.loads(paper)
    try:
        date = int(paper['update_date'].split('-')[0])
        if date > 2019:
            dataframe['title'].append(paper['title'])
            dataframe['year'].append(date)
            dataframe['abstract'].append(paper['abstract'])
    except: pass

In [4]:
df = pd.DataFrame(dataframe)
df.head(10)

,title,year,abstract
0,Convergence of the discrete dipole approximati...,2022,We performed a rigorous theoretical converge...
1,The discrete dipole approximation: an overview...,2022,We present a review of the discrete dipole a...
2,Mapping radii of metric spaces,2021,It is known that every closed curve of lengt...
3,What can emission lines tell us?,2023,1 Generalities\n 2 Empirical diagnostics ba...
4,The affine part of the Picard scheme,2021,We describe the maximal torus and maximal un...
5,Manifolds admitting a $\tilde G_2$-structure,2023,We find a necessary and sufficient condition...
6,Actions for the Bosonic String with the Curved...,2020,At first we introduce an action for the stri...
7,Stochastic action principle and maximum entropy,2020,A stochastic action principle for stochastic...
8,Axiom A polynomial skew products of C^2 and th...,2023,A polynomial skew product of C^2 is a map of...
9,Depletion effects in smectic phases of hard ro...,2022,It is known that when hard spheres are added...


In [5]:
del dataframe
print(f"Samples: {df.shape[0]}\nFeatures: {df.shape[1]}")

Samples: 826051
Features: 3


In [6]:
any(df.isna().sum())
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 826051 entries, 0 to 826050
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   title     826051 non-null  object
 1   year      826051 non-null  int64 
 2   abstract  826051 non-null  object
dtypes: int64(1), object(2)
memory usage: 18.9+ MB


In [7]:
df['length'] = df['abstract'].str.len()
df.head(10)

,title,year,abstract,length
0,Convergence of the discrete dipole approximati...,2022,We performed a rigorous theoretical converge...,832
1,The discrete dipole approximation: an overview...,2022,We present a review of the discrete dipole a...,617
2,Mapping radii of metric spaces,2021,It is known that every closed curve of lengt...,737
3,What can emission lines tell us?,2023,1 Generalities\n 2 Empirical diagnostics ba...,181
4,The affine part of the Picard scheme,2021,We describe the maximal torus and maximal un...,126
5,Manifolds admitting a $\tilde G_2$-structure,2023,We find a necessary and sufficient condition...,220
6,Actions for the Bosonic String with the Curved...,2020,At first we introduce an action for the stri...,414
7,Stochastic action principle and maximum entropy,2020,A stochastic action principle for stochastic...,770
8,Axiom A polynomial skew products of C^2 and th...,2023,A polynomial skew product of C^2 is a map of...,784
9,Depletion effects in smectic phases of hard ro...,2022,It is known that when hard spheres are added...,1808


In [8]:
def word_count(x):
    return len(x.split())

df['word_count'] = df['abstract'].apply(word_count)
df.head()

,title,year,abstract,length,word_count
0,Convergence of the discrete dipole approximati...,2022,We performed a rigorous theoretical converge...,832,133
1,The discrete dipole approximation: an overview...,2022,We present a review of the discrete dipole a...,617,96
2,Mapping radii of metric spaces,2021,It is known that every closed curve of lengt...,737,138
3,What can emission lines tell us?,2023,1 Generalities\n 2 Empirical diagnostics ba...,181,27
4,The affine part of the Picard scheme,2021,We describe the maximal torus and maximal un...,126,21


In [9]:
print(f"Mean of Word Count: {df['word_count'].mean():.2f}\nMedian of Word Count: {df['word_count'].median()}")

Mean of Word Count: 156.55
Median of Word Count: 154.0


In [10]:
MODEL_URL = "https://www.kaggle.com/models/google/universal-sentence-encoder/frameworks/TensorFlow2/variations/universal-sentence-encoder/versions/2"
sentence_encoder_layer = hub.KerasLayer(MODEL_URL,
                                        input_shape=[],
                                        dtype=tf.string,
                                        trainable=False,
                                        name="use")
abstracts = df["abstract"].to_list()

In [11]:
# Setup for embeddings computation
embeddings = []
batch_size = 300
num_batches = len(abstracts) // batch_size

In [12]:
# Compute Embeddings in batches
for i in range(num_batches):
    batch_abstracts = abstracts[i*batch_size : (i+1)*batch_size]
    batch_embeddings = sentence_encoder_layer(batch_abstracts)
    embeddings.extend(batch_embeddings.numpy())

# Embeddings for remaining abstracts
remaining_abstracts = abstracts[num_batches*batch_size:]
if len(remaining_abstracts) > 0:
    remaining_embeddings = sentence_encoder_layer(remaining_abstracts)
    embeddings.extend(remaining_embeddings.numpy())
    
embeddings = np.array(embeddings)
y = df.index

In [13]:
nn = KNeighborsClassifier(n_neighbors=6)
nn.fit(embeddings,y)

KNeighborsClassifier(n_neighbors=6)

In [1]:
def calculate_user_embedding(title, abstract):
    user_abstract = sentence_encoder_layer([title, abstract])
    print(user_abstract.numpy())
    return user_abstract.numpy()


def find_similar_papers(user_embeddings, k=5):
    dist, indices = nn.kneighbors(X=user_embeddings, n_neighbors=k)
    print(indices[0])
    return indices[0]

In [ ]:
model_info = {
    "model": nn,
    "embeddings": embeddings.tolist(),
}

with open("model.pkl", 'wb') as file:
    dill.dump(model_info, file)
df.to_csv("data.csv", index=False)

In [2]:
user_title = "Current Topological and Machine Learning Applications for Bias Detection in Text"
user_abstract = "Institutional bias can impact patient outcomes, educational attainment, and legal system navigation. Written records often reflect bias, and once bias is identified; it is possible to refer individuals for training to reduce bias. Many machine learning tools exist to explore text data and create predictive models that can search written records to identify real-time bias. However, few previous studies investigate large language model embeddings and geometric models of biased text data to understand geometry's impact on bias modeling accuracy. To overcome this issue, this study utilizes the RedditBias database to analyze textual biases. Four transformer models, including BERT and RoBERTa variants, were explored. Post-embedding, t-SNE allowed two-dimensional visualization of data. KNN classifiers differentiated bias types, with lower k-values proving more effective. Findings suggest BERT, particularly mini BERT, excels in bias classification, while multilingual models lag. The recommendation emphasizes refining monolingual models and exploring domain-specific biases."
user_embeddings = calculate_user_embedding(user_title, user_abstract)
similar_paper_indices = find_similar_papers(user_embeddings)
for i, idx in enumerate(similar_paper_indices):
    recommended_paper = df['title'][idx]
    print(f"Recommendation {i + 1}:\n{recommended_paper}\n")

NameError: name 'sentence_encoder_layer' is not defined